In [1]:
import pyspark

In [2]:
from pyspark.sql import SparkSession

In [3]:
#loading the required sql functions for the further processing
from pyspark.sql import functions
from pyspark.sql.functions import col, trim, lower, regexp_replace

In [4]:
spark = SparkSession.builder \
    .appName("BookRecommendEDA") \
    .getOrCreate()

25/06/28 18:41:24 WARN Utils: Your hostname, vaibhavi-HP-Laptop-15-fd0xxx resolves to a loopback address: 127.0.1.1; using 192.168.0.107 instead (on interface wlo1)
25/06/28 18:41:24 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/28 18:41:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [7]:
df_reviews = spark.read.csv("/home/vaibhavi/spark-ml-venv/ml_project/data/Books_rating.csv", header=True, inferSchema=True)
df_reviews.printSchema()
df_reviews.show(5)

root
 |-- Id: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- User_id: string (nullable = true)
 |-- profileName: string (nullable = true)
 |-- review/helpfulness: string (nullable = true)
 |-- review/score: string (nullable = true)
 |-- review/time: string (nullable = true)
 |-- review/summary: string (nullable = true)
 |-- review/text: string (nullable = true)

+----------+--------------------+-----+--------------+--------------------+------------------+------------+-----------+--------------------+--------------------+
|        Id|               Title|Price|       User_id|         profileName|review/helpfulness|review/score|review/time|      review/summary|         review/text|
+----------+--------------------+-----+--------------+--------------------+------------------+------------+-----------+--------------------+--------------------+
|1882931173|Its Only Art If I...| NULL| AVCGYZL8FQQTD|"Jim of Oz ""jim-...|               7/

In [8]:
df_books = spark.read.csv("/home/vaibhavi/spark-ml-venv/ml_project/data/books_data.csv", header=True, inferSchema=True)
df_books.printSchema()
df_books.show(5)

root
 |-- Title: string (nullable = true)
 |-- description: string (nullable = true)
 |-- authors: string (nullable = true)
 |-- image: string (nullable = true)
 |-- previewLink: string (nullable = true)
 |-- publisher: string (nullable = true)
 |-- publishedDate: string (nullable = true)
 |-- infoLink: string (nullable = true)
 |-- categories: string (nullable = true)
 |-- ratingsCount: string (nullable = true)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------+--------------------+--------------------+------------+
|               Title|         description|             authors|               image|         previewLink|           publisher| publishedDate|            infoLink|          categories|ratingsCount|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------+--------------------+--------------------+---

actual Schema : Title,description,authors,image,previewLink,publisher,publishedDate,infoLink,categories,ratingsCount

#required columns from both the tables:
- title, userId, review_score, tb2 - title, authors ,publisher, categories,description, ratingsCount

In [ ]:
#selecting required columns from books dataframe
df_required1 = df_books.select("Title","description","authors","publisher", "categories","ratingsCount")

In [ ]:
#selecting required columns from reviews dataframe
df_required2 = df_reviews.select("Title","User_id","review/score")

In [13]:
df_required1.printSchema()

root
 |-- Title: string (nullable = true)
 |-- description: string (nullable = true)
 |-- authors: string (nullable = true)
 |-- publisher: string (nullable = true)
 |-- categories: string (nullable = true)
 |-- ratingsCount: string (nullable = true)



In [14]:
df_required2.printSchema()

root
 |-- Title: string (nullable = true)
 |-- User_id: string (nullable = true)
 |-- review/score: string (nullable = true)



In [22]:
#df_final = df_required1.join(df_required2, df_required1.Title == df_required2.Title,"cross")

#reviews per title
count = df_required2.groupBy("Title").count().orderBy("count").show()
print(count)

+--------------------+-----+
|               Title|count|
+--------------------+-----+
|  tales of chinatown|    1|
|"Buffy contre les...|    1|
|The Dum Dum Bulle...|    1|
|The Poll-ette Hos...|    1|
|Adviser's guide t...|    1|
|Fast Forward Lead...|    1|
|A Groovy Guide to...|    1|
|Iso 9001:2000 Ach...|    1|
|The Gesalt Therap...|    1|
|Boys' life of Chr...|    1|
|Maine Lingo: Boil...|    1|
|Sir Marrok;: A ta...|    1|
|  Tus Zonas Sagradas|    1|
|All Fall Down Pb ...|    1|
| English in Action 1|    1|
|What Mrs. McGilli...|    1|
|A Voice of Thunde...|    1|
|           Bluebeard|    1|
|Variational metho...|    1|
|Sodium Calories &...|    1|
+--------------------+-----+
only showing top 20 rows

None


In [21]:
#distinct titles:
distinct_tiles = df_required2.select("Title").distinct().count()
print(distinct_tiles)

212400


In [23]:
distinct_tiles_2 = df_required1.select("Title").distinct().count()
print(distinct_tiles_2)

212400


In [25]:
total_count = df_required2.count()
print(total_count)

3000000


In [27]:
total_count_reviews = df_required1.count()
print(total_count_reviews)

212404
